# Next Best Product Model - V1

**Goal:** Predict which members are most likely to open a Credit Card in the next 12 months.

**Approach:** Look-alike product adoption - members with similar profiles tend to add similar products.

**What this notebook does:**
1. Creates realistic dummy data for 5,000 credit union members
2. Aggregates features (product holdings, transaction behavior, demographics)
3. Trains a simple Logistic Regression model (interpretable)
4. Trains an XGBoost model (potentially more accurate)
5. Compares both models and shows feature importances
6. Saves the model to Snowflake Model Registry

## Step 1: Create Raw Data Tables
We'll create 4 tables: MEMBERS, ACCOUNTS, PRODUCTS, TRANSACTIONS

In [ ]:
CREATE OR REPLACE DATABASE NEXT_BEST_PRODUCT_DEMO;
CREATE OR REPLACE SCHEMA RAW_DATA;
USE SCHEMA NEXT_BEST_PRODUCT_DEMO.RAW_DATA;

In [ ]:
-- MEMBERS table: 5000 credit union members with demographics
CREATE OR REPLACE TABLE MEMBERS AS
WITH member_base AS (
    SELECT 
        ROW_NUMBER() OVER (ORDER BY SEQ4()) AS MEMBER_ID,
        DATEADD(day, -UNIFORM(365, 7300, RANDOM()), '2024-01-01'::DATE) AS JOIN_DATE,
        UNIFORM(18, 85, RANDOM()) AS AGE,
        CASE UNIFORM(1, 10, RANDOM())
            WHEN 1 THEN 'Under 25K'
            WHEN 2 THEN 'Under 25K'
            WHEN 3 THEN '25K-50K'
            WHEN 4 THEN '25K-50K'
            WHEN 5 THEN '25K-50K'
            WHEN 6 THEN '50K-75K'
            WHEN 7 THEN '50K-75K'
            WHEN 8 THEN '75K-100K'
            WHEN 9 THEN '100K-150K'
            ELSE '150K+'
        END AS INCOME_BRACKET,
        CASE UNIFORM(1, 5, RANDOM())
            WHEN 1 THEN 'Single'
            WHEN 2 THEN 'Married'
            WHEN 3 THEN 'Married'
            WHEN 4 THEN 'Divorced'
            ELSE 'Widowed'
        END AS MARITAL_STATUS,
        ROUND(UNIFORM(-200, 1500, RANDOM()) + 
              CASE 
                  WHEN UNIFORM(1, 10, RANDOM()) <= 3 THEN UNIFORM(500, 2000, RANDOM())
                  ELSE 0
              END, 2) AS CURRENT_PROFITABILITY
    FROM TABLE(GENERATOR(ROWCOUNT => 5000))
)
SELECT * FROM member_base;

In [ ]:
-- ACCOUNTS table: checking, savings, money market accounts with balances
CREATE OR REPLACE TABLE ACCOUNTS AS
WITH account_types AS (
    SELECT m.MEMBER_ID,
           'CHECKING' AS ACCOUNT_TYPE,
           ROUND(UNIFORM(100, 15000, RANDOM()) * 
                 CASE m.INCOME_BRACKET
                     WHEN 'Under 25K' THEN 0.3
                     WHEN '25K-50K' THEN 0.5
                     WHEN '50K-75K' THEN 0.8
                     WHEN '75K-100K' THEN 1.2
                     WHEN '100K-150K' THEN 1.8
                     ELSE 2.5
                 END, 2) AS BALANCE,
           m.JOIN_DATE AS OPEN_DATE
    FROM MEMBERS m
    WHERE UNIFORM(1, 100, RANDOM()) <= 95  -- 95% have checking
    
    UNION ALL
    
    SELECT m.MEMBER_ID,
           'SAVINGS' AS ACCOUNT_TYPE,
           ROUND(UNIFORM(500, 50000, RANDOM()) * 
                 CASE m.INCOME_BRACKET
                     WHEN 'Under 25K' THEN 0.2
                     WHEN '25K-50K' THEN 0.4
                     WHEN '50K-75K' THEN 0.7
                     WHEN '75K-100K' THEN 1.0
                     WHEN '100K-150K' THEN 1.5
                     ELSE 2.0
                 END, 2) AS BALANCE,
           DATEADD(day, UNIFORM(0, 365, RANDOM()), m.JOIN_DATE) AS OPEN_DATE
    FROM MEMBERS m
    WHERE UNIFORM(1, 100, RANDOM()) <= 80  -- 80% have savings
    
    UNION ALL
    
    SELECT m.MEMBER_ID,
           'MONEY_MARKET' AS ACCOUNT_TYPE,
           ROUND(UNIFORM(5000, 100000, RANDOM()) * 
                 CASE m.INCOME_BRACKET
                     WHEN 'Under 25K' THEN 0.1
                     WHEN '25K-50K' THEN 0.3
                     WHEN '50K-75K' THEN 0.5
                     WHEN '75K-100K' THEN 0.8
                     WHEN '100K-150K' THEN 1.2
                     ELSE 1.8
                 END, 2) AS BALANCE,
           DATEADD(day, UNIFORM(0, 730, RANDOM()), m.JOIN_DATE) AS OPEN_DATE
    FROM MEMBERS m
    WHERE UNIFORM(1, 100, RANDOM()) <= 25  -- 25% have money market
      AND m.INCOME_BRACKET IN ('75K-100K', '100K-150K', '150K+')
)
SELECT 
    ROW_NUMBER() OVER (ORDER BY MEMBER_ID, ACCOUNT_TYPE) AS ACCOUNT_ID,
    MEMBER_ID,
    ACCOUNT_TYPE,
    BALANCE,
    OPEN_DATE
FROM account_types;

In [ ]:
-- PRODUCTS table: credit cards, auto loans, personal loans, mortgages
CREATE OR REPLACE TABLE PRODUCTS AS
WITH product_base AS (
    -- Credit Cards: higher likelihood for higher income, good profitability
    SELECT m.MEMBER_ID,
           'CREDIT_CARD' AS PRODUCT_TYPE,
           DATEADD(day, UNIFORM(30, 1500, RANDOM()), m.JOIN_DATE) AS OPEN_DATE,
           CASE 
               WHEN m.INCOME_BRACKET IN ('100K-150K', '150K+') THEN UNIFORM(10000, 25000, RANDOM())
               WHEN m.INCOME_BRACKET IN ('75K-100K') THEN UNIFORM(5000, 15000, RANDOM())
               WHEN m.INCOME_BRACKET IN ('50K-75K') THEN UNIFORM(3000, 10000, RANDOM())
               ELSE UNIFORM(1000, 5000, RANDOM())
           END AS CREDIT_LIMIT
    FROM MEMBERS m
    WHERE UNIFORM(1, 100, RANDOM()) <= 
          CASE 
              WHEN m.INCOME_BRACKET IN ('100K-150K', '150K+') AND m.CURRENT_PROFITABILITY > 500 THEN 70
              WHEN m.INCOME_BRACKET IN ('75K-100K') AND m.CURRENT_PROFITABILITY > 200 THEN 55
              WHEN m.INCOME_BRACKET IN ('50K-75K') AND m.CURRENT_PROFITABILITY > 0 THEN 40
              WHEN m.INCOME_BRACKET IN ('25K-50K') THEN 25
              ELSE 15
          END
      AND m.AGE >= 21
    
    UNION ALL
    
    -- Auto Loans
    SELECT m.MEMBER_ID,
           'AUTO_LOAN' AS PRODUCT_TYPE,
           DATEADD(day, UNIFORM(60, 2000, RANDOM()), m.JOIN_DATE) AS OPEN_DATE,
           CASE 
               WHEN m.INCOME_BRACKET IN ('100K-150K', '150K+') THEN UNIFORM(25000, 60000, RANDOM())
               WHEN m.INCOME_BRACKET IN ('75K-100K') THEN UNIFORM(20000, 45000, RANDOM())
               WHEN m.INCOME_BRACKET IN ('50K-75K') THEN UNIFORM(15000, 35000, RANDOM())
               ELSE UNIFORM(8000, 25000, RANDOM())
           END AS CREDIT_LIMIT
    FROM MEMBERS m
    WHERE UNIFORM(1, 100, RANDOM()) <= 
          CASE 
              WHEN m.AGE BETWEEN 25 AND 55 AND m.INCOME_BRACKET IN ('75K-100K', '100K-150K', '150K+') THEN 35
              WHEN m.AGE BETWEEN 25 AND 55 AND m.INCOME_BRACKET IN ('50K-75K') THEN 25
              WHEN m.AGE BETWEEN 18 AND 35 THEN 20
              ELSE 12
          END
      AND m.AGE >= 18
    
    UNION ALL
    
    -- Personal Loans
    SELECT m.MEMBER_ID,
           'PERSONAL_LOAN' AS PRODUCT_TYPE,
           DATEADD(day, UNIFORM(90, 1800, RANDOM()), m.JOIN_DATE) AS OPEN_DATE,
           UNIFORM(2000, 25000, RANDOM()) AS CREDIT_LIMIT
    FROM MEMBERS m
    WHERE UNIFORM(1, 100, RANDOM()) <= 18
      AND m.AGE >= 21
    
    UNION ALL
    
    -- Mortgages (home equity lines)
    SELECT m.MEMBER_ID,
           'MORTGAGE' AS PRODUCT_TYPE,
           DATEADD(day, UNIFORM(365, 3000, RANDOM()), m.JOIN_DATE) AS OPEN_DATE,
           UNIFORM(50000, 400000, RANDOM()) AS CREDIT_LIMIT
    FROM MEMBERS m
    WHERE UNIFORM(1, 100, RANDOM()) <= 
          CASE 
              WHEN m.AGE BETWEEN 30 AND 60 AND m.INCOME_BRACKET IN ('100K-150K', '150K+') THEN 30
              WHEN m.AGE BETWEEN 30 AND 60 AND m.INCOME_BRACKET IN ('75K-100K') THEN 20
              WHEN m.AGE BETWEEN 30 AND 60 AND m.INCOME_BRACKET IN ('50K-75K') THEN 12
              ELSE 5
          END
      AND m.AGE >= 25
      AND m.MARITAL_STATUS IN ('Married', 'Divorced', 'Widowed')
)
SELECT 
    ROW_NUMBER() OVER (ORDER BY MEMBER_ID, PRODUCT_TYPE) AS PRODUCT_ID,
    MEMBER_ID,
    PRODUCT_TYPE,
    OPEN_DATE,
    CREDIT_LIMIT
FROM product_base
WHERE OPEN_DATE <= '2025-01-01';

In [ ]:
-- TRANSACTIONS table: realistic transaction patterns
CREATE OR REPLACE TABLE TRANSACTIONS AS
WITH txn_base AS (
    SELECT 
        m.MEMBER_ID,
        DATEADD(day, UNIFORM(0, 730, RANDOM()), '2023-01-01'::DATE) AS TRANSACTION_DATE,
        CASE UNIFORM(1, 20, RANDOM())
            WHEN 1 THEN 'DIRECT_DEPOSIT'
            WHEN 2 THEN 'DIRECT_DEPOSIT'
            WHEN 3 THEN 'DIRECT_DEPOSIT'
            WHEN 4 THEN 'DEBIT_CARD'
            WHEN 5 THEN 'DEBIT_CARD'
            WHEN 6 THEN 'DEBIT_CARD'
            WHEN 7 THEN 'DEBIT_CARD'
            WHEN 8 THEN 'DEBIT_CARD'
            WHEN 9 THEN 'ACH_PAYMENT'
            WHEN 10 THEN 'ACH_PAYMENT'
            WHEN 11 THEN 'TRANSFER'
            WHEN 12 THEN 'TRANSFER'
            WHEN 13 THEN 'ATM_WITHDRAWAL'
            WHEN 14 THEN 'ATM_WITHDRAWAL'
            WHEN 15 THEN 'CHECK'
            WHEN 16 THEN 'BILL_PAY'
            WHEN 17 THEN 'BILL_PAY'
            WHEN 18 THEN 'WIRE'
            WHEN 19 THEN 'FEE'
            ELSE 'OTHER'
        END AS TRANSACTION_TYPE,
        CASE 
            WHEN UNIFORM(1, 20, RANDOM()) <= 3 THEN  -- Direct deposits
                ROUND(UNIFORM(1500, 8000, RANDOM()) * 
                      CASE m.INCOME_BRACKET
                          WHEN 'Under 25K' THEN 0.4
                          WHEN '25K-50K' THEN 0.6
                          WHEN '50K-75K' THEN 0.9
                          WHEN '75K-100K' THEN 1.2
                          WHEN '100K-150K' THEN 1.6
                          ELSE 2.2
                      END, 2)
            ELSE  -- Regular transactions
                ROUND(-1 * UNIFORM(5, 500, RANDOM()) * 
                      CASE m.INCOME_BRACKET
                          WHEN 'Under 25K' THEN 0.5
                          WHEN '25K-50K' THEN 0.7
                          WHEN '50K-75K' THEN 1.0
                          WHEN '75K-100K' THEN 1.3
                          WHEN '100K-150K' THEN 1.6
                          ELSE 2.0
                      END, 2)
        END AS AMOUNT,
        seq AS txn_seq
    FROM MEMBERS m,
         LATERAL (
             SELECT SEQ4() as seq 
             FROM TABLE(GENERATOR(ROWCOUNT => 
                 CASE 
                     WHEN m.INCOME_BRACKET IN ('100K-150K', '150K+') THEN UNIFORM(80, 200, RANDOM())
                     WHEN m.INCOME_BRACKET IN ('75K-100K') THEN UNIFORM(60, 150, RANDOM())
                     WHEN m.INCOME_BRACKET IN ('50K-75K') THEN UNIFORM(40, 120, RANDOM())
                     ELSE UNIFORM(20, 80, RANDOM())
                 END
             ))
         )
)
SELECT 
    ROW_NUMBER() OVER (ORDER BY MEMBER_ID, TRANSACTION_DATE, txn_seq) AS TRANSACTION_ID,
    MEMBER_ID,
    TRANSACTION_DATE,
    TRANSACTION_TYPE,
    AMOUNT,
    CASE WHEN TRANSACTION_TYPE = 'DIRECT_DEPOSIT' THEN 1 ELSE 0 END AS IS_DIRECT_DEPOSIT
FROM txn_base
WHERE TRANSACTION_DATE <= '2025-01-01';

In [ ]:
-- Verify data counts
SELECT 'MEMBERS' as TABLE_NAME, COUNT(*) as ROW_COUNT FROM MEMBERS
UNION ALL
SELECT 'ACCOUNTS', COUNT(*) FROM ACCOUNTS
UNION ALL
SELECT 'PRODUCTS', COUNT(*) FROM PRODUCTS
UNION ALL
SELECT 'TRANSACTIONS', COUNT(*) FROM TRANSACTIONS;

## Step 2: Create Feature Table
Aggregate features for each member: product holdings, transaction behavior, demographic info

In [ ]:
CREATE OR REPLACE SCHEMA NEXT_BEST_PRODUCT_DEMO.FEATURES;
USE SCHEMA NEXT_BEST_PRODUCT_DEMO.FEATURES;

In [ ]:
-- Feature table with snapshot date of 2024-01-01
-- We use data BEFORE this date to predict who will open a credit card AFTER
CREATE OR REPLACE TABLE MEMBER_FEATURES AS
WITH snapshot AS (SELECT '2024-01-01'::DATE AS snapshot_date),

-- Product holdings as of snapshot date
product_features AS (
    SELECT 
        m.MEMBER_ID,
        MAX(CASE WHEN p.PRODUCT_TYPE = 'CREDIT_CARD' AND p.OPEN_DATE < s.snapshot_date THEN 1 ELSE 0 END) AS HAS_CREDIT_CARD,
        MAX(CASE WHEN p.PRODUCT_TYPE = 'AUTO_LOAN' AND p.OPEN_DATE < s.snapshot_date THEN 1 ELSE 0 END) AS HAS_AUTO_LOAN,
        MAX(CASE WHEN p.PRODUCT_TYPE = 'PERSONAL_LOAN' AND p.OPEN_DATE < s.snapshot_date THEN 1 ELSE 0 END) AS HAS_PERSONAL_LOAN,
        MAX(CASE WHEN p.PRODUCT_TYPE = 'MORTGAGE' AND p.OPEN_DATE < s.snapshot_date THEN 1 ELSE 0 END) AS HAS_MORTGAGE,
        COUNT(DISTINCT CASE WHEN p.OPEN_DATE < s.snapshot_date THEN p.PRODUCT_TYPE END) AS NUM_PRODUCTS
    FROM RAW_DATA.MEMBERS m
    CROSS JOIN snapshot s
    LEFT JOIN RAW_DATA.PRODUCTS p ON m.MEMBER_ID = p.MEMBER_ID
    GROUP BY m.MEMBER_ID
),

-- Account features
account_features AS (
    SELECT 
        m.MEMBER_ID,
        MAX(CASE WHEN a.ACCOUNT_TYPE = 'CHECKING' AND a.OPEN_DATE < s.snapshot_date THEN 1 ELSE 0 END) AS HAS_CHECKING,
        MAX(CASE WHEN a.ACCOUNT_TYPE = 'SAVINGS' AND a.OPEN_DATE < s.snapshot_date THEN 1 ELSE 0 END) AS HAS_SAVINGS,
        MAX(CASE WHEN a.ACCOUNT_TYPE = 'MONEY_MARKET' AND a.OPEN_DATE < s.snapshot_date THEN 1 ELSE 0 END) AS HAS_MONEY_MARKET,
        COALESCE(SUM(CASE WHEN a.OPEN_DATE < s.snapshot_date THEN a.BALANCE ELSE 0 END), 0) AS TOTAL_BALANCE,
        COALESCE(AVG(CASE WHEN a.OPEN_DATE < s.snapshot_date THEN a.BALANCE END), 0) AS AVG_BALANCE
    FROM RAW_DATA.MEMBERS m
    CROSS JOIN snapshot s
    LEFT JOIN RAW_DATA.ACCOUNTS a ON m.MEMBER_ID = a.MEMBER_ID
    GROUP BY m.MEMBER_ID
),

-- Transaction features (last 6 months before snapshot)
txn_features AS (
    SELECT 
        m.MEMBER_ID,
        COUNT(t.TRANSACTION_ID) AS TXN_COUNT_6M,
        COALESCE(SUM(ABS(t.AMOUNT)), 0) AS TXN_VOLUME_6M,
        COALESCE(AVG(ABS(t.AMOUNT)), 0) AS AVG_TXN_AMOUNT,
        MAX(t.IS_DIRECT_DEPOSIT) AS HAS_DIRECT_DEPOSIT,
        COALESCE(SUM(CASE WHEN t.IS_DIRECT_DEPOSIT = 1 THEN t.AMOUNT ELSE 0 END), 0) AS DIRECT_DEPOSIT_TOTAL,
        COUNT(CASE WHEN t.TRANSACTION_TYPE = 'DEBIT_CARD' THEN 1 END) AS DEBIT_CARD_TXN_COUNT
    FROM RAW_DATA.MEMBERS m
    CROSS JOIN snapshot s
    LEFT JOIN RAW_DATA.TRANSACTIONS t 
        ON m.MEMBER_ID = t.MEMBER_ID
        AND t.TRANSACTION_DATE >= DATEADD(month, -6, s.snapshot_date)
        AND t.TRANSACTION_DATE < s.snapshot_date
    GROUP BY m.MEMBER_ID
)

SELECT 
    m.MEMBER_ID,
    
    -- Demographics
    m.AGE,
    CASE m.INCOME_BRACKET
        WHEN 'Under 25K' THEN 1
        WHEN '25K-50K' THEN 2
        WHEN '50K-75K' THEN 3
        WHEN '75K-100K' THEN 4
        WHEN '100K-150K' THEN 5
        ELSE 6
    END AS INCOME_LEVEL,
    CASE m.MARITAL_STATUS
        WHEN 'Single' THEN 0
        WHEN 'Married' THEN 1
        WHEN 'Divorced' THEN 2
        ELSE 3
    END AS MARITAL_STATUS_CODE,
    DATEDIFF(month, m.JOIN_DATE, s.snapshot_date) AS TENURE_MONTHS,
    m.CURRENT_PROFITABILITY,
    
    -- Account features
    af.HAS_CHECKING,
    af.HAS_SAVINGS,
    af.HAS_MONEY_MARKET,
    af.TOTAL_BALANCE,
    af.AVG_BALANCE,
    
    -- Product features (exclude credit card since that's what we're predicting)
    pf.HAS_CREDIT_CARD,
    pf.HAS_AUTO_LOAN,
    pf.HAS_PERSONAL_LOAN,
    pf.HAS_MORTGAGE,
    pf.NUM_PRODUCTS,
    
    -- Transaction features
    tf.TXN_COUNT_6M,
    tf.TXN_VOLUME_6M,
    tf.AVG_TXN_AMOUNT,
    COALESCE(tf.HAS_DIRECT_DEPOSIT, 0) AS HAS_DIRECT_DEPOSIT,
    tf.DIRECT_DEPOSIT_TOTAL,
    tf.DEBIT_CARD_TXN_COUNT
    
FROM RAW_DATA.MEMBERS m
CROSS JOIN snapshot s
LEFT JOIN product_features pf ON m.MEMBER_ID = pf.MEMBER_ID
LEFT JOIN account_features af ON m.MEMBER_ID = af.MEMBER_ID
LEFT JOIN txn_features tf ON m.MEMBER_ID = tf.MEMBER_ID
WHERE m.JOIN_DATE < s.snapshot_date;  -- Only include members who joined before snapshot

In [ ]:
SELECT * FROM MEMBER_FEATURES LIMIT 10;

## Step 3: Create Labels
Label = 1 if member opened a credit card between 2024-01-01 and 2025-01-01

In [ ]:
-- Label table: did member open a credit card in the 12 months after snapshot?
CREATE OR REPLACE TABLE CREDIT_CARD_LABELS AS
WITH snapshot AS (SELECT '2024-01-01'::DATE AS snapshot_date)
SELECT 
    f.MEMBER_ID,
    CASE 
        WHEN EXISTS (
            SELECT 1 
            FROM RAW_DATA.PRODUCTS p
            CROSS JOIN snapshot s
            WHERE p.MEMBER_ID = f.MEMBER_ID
              AND p.PRODUCT_TYPE = 'CREDIT_CARD'
              AND p.OPEN_DATE >= s.snapshot_date
              AND p.OPEN_DATE < DATEADD(month, 12, s.snapshot_date)
        ) THEN 1
        ELSE 0
    END AS OPENED_CREDIT_CARD_12M
FROM MEMBER_FEATURES f
WHERE f.HAS_CREDIT_CARD = 0;  -- Only members who didn't have a credit card at snapshot

In [ ]:
-- Check label distribution
SELECT 
    OPENED_CREDIT_CARD_12M,
    COUNT(*) as COUNT,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as PERCENTAGE
FROM CREDIT_CARD_LABELS
GROUP BY OPENED_CREDIT_CARD_12M;

In [ ]:
-- Create final training dataset
CREATE OR REPLACE TABLE TRAINING_DATA AS
SELECT 
    f.MEMBER_ID,
    f.AGE,
    f.INCOME_LEVEL,
    f.MARITAL_STATUS_CODE,
    f.TENURE_MONTHS,
    f.CURRENT_PROFITABILITY,
    f.HAS_CHECKING,
    f.HAS_SAVINGS,
    f.HAS_MONEY_MARKET,
    f.TOTAL_BALANCE,
    f.AVG_BALANCE,
    f.HAS_AUTO_LOAN,
    f.HAS_PERSONAL_LOAN,
    f.HAS_MORTGAGE,
    f.NUM_PRODUCTS,
    f.TXN_COUNT_6M,
    f.TXN_VOLUME_6M,
    f.AVG_TXN_AMOUNT,
    f.HAS_DIRECT_DEPOSIT,
    f.DIRECT_DEPOSIT_TOTAL,
    f.DEBIT_CARD_TXN_COUNT,
    l.OPENED_CREDIT_CARD_12M
FROM MEMBER_FEATURES f
INNER JOIN CREDIT_CARD_LABELS l ON f.MEMBER_ID = l.MEMBER_ID;

In [ ]:
SELECT COUNT(*) as TRAINING_ROWS FROM TRAINING_DATA;

## Step 4: Train Logistic Regression Model
Simple, interpretable model that's easy to explain to stakeholders

In [ ]:
from snowflake.snowpark.context import get_active_session
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix

session = get_active_session()

In [ ]:
df = session.table("NEXT_BEST_PRODUCT_DEMO.FEATURES.TRAINING_DATA").to_pandas()
print(f"Training data shape: {df.shape}")
df.head()

In [ ]:
feature_columns = [
    'AGE', 'INCOME_LEVEL', 'MARITAL_STATUS_CODE', 'TENURE_MONTHS', 'CURRENT_PROFITABILITY',
    'HAS_CHECKING', 'HAS_SAVINGS', 'HAS_MONEY_MARKET', 'TOTAL_BALANCE', 'AVG_BALANCE',
    'HAS_AUTO_LOAN', 'HAS_PERSONAL_LOAN', 'HAS_MORTGAGE', 'NUM_PRODUCTS',
    'TXN_COUNT_6M', 'TXN_VOLUME_6M', 'AVG_TXN_AMOUNT', 'HAS_DIRECT_DEPOSIT',
    'DIRECT_DEPOSIT_TOTAL', 'DEBIT_CARD_TXN_COUNT'
]

X = df[feature_columns]
y = df['OPENED_CREDIT_CARD_12M']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} rows")
print(f"Test set: {X_test.shape[0]} rows")
print(f"\nTarget distribution in training set:")
print(y_train.value_counts(normalize=True))

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
model.fit(X_train_scaled, y_train)

print("Model trained successfully!")

In [ ]:
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_proba)

print("=" * 50)
print("MODEL PERFORMANCE METRICS")
print("=" * 50)
print(f"Accuracy:  {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"AUC-ROC:   {auc_roc:.3f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

## Step 4b: Train XGBoost Model (Alternative)
XGBoost often provides better accuracy but is less interpretable than Logistic Regression

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),
    use_label_encoder=False,
    eval_metric='logloss'
)

xgb_model.fit(X_train, y_train)
print("XGBoost model trained successfully!")

In [ ]:
y_pred_xgb = xgb_model.predict(X_test)
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
auc_roc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)

print("=" * 50)
print("XGBOOST MODEL PERFORMANCE")
print("=" * 50)
print(f"Accuracy:  {accuracy_xgb:.3f}")
print(f"Precision: {precision_xgb:.3f}")
print(f"Recall:    {recall_xgb:.3f}")
print(f"AUC-ROC:   {auc_roc_xgb:.3f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb))

In [ ]:
xgb_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("=" * 50)
print("XGBOOST TOP 10 FEATURE IMPORTANCES")
print("=" * 50)
print("\nHigher importance = more influence on predictions\n")
print(xgb_importance.head(10).to_string(index=False))

In [ ]:
comparison = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'AUC-ROC'],
    'Logistic Regression': [accuracy, precision, recall, auc_roc],
    'XGBoost': [accuracy_xgb, precision_xgb, recall_xgb, auc_roc_xgb]
})
comparison['Difference'] = comparison['XGBoost'] - comparison['Logistic Regression']
comparison['Winner'] = comparison['Difference'].apply(lambda x: 'XGBoost' if x > 0 else 'Logistic Regression' if x < 0 else 'Tie')

print("=" * 70)
print("MODEL COMPARISON: Logistic Regression vs XGBoost")
print("=" * 70)
print(comparison.to_string(index=False))
print("\n" + "=" * 70)
print("RECOMMENDATION:")
print("=" * 70)
if auc_roc_xgb > auc_roc + 0.02:
    print("XGBoost shows meaningful improvement. Consider using it if interpretability is less critical.")
elif auc_roc > auc_roc_xgb + 0.02:
    print("Logistic Regression performs better. Stick with the simpler, more interpretable model.")
else:
    print("Both models perform similarly. Logistic Regression is recommended for its interpretability.")

In [ ]:
feature_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Coefficient': model.coef_[0],
    'Abs_Coefficient': abs(model.coef_[0])
}).sort_values('Abs_Coefficient', ascending=False)

print("\n" + "=" * 50)
print("TOP 10 FEATURE IMPORTANCES")
print("=" * 50)
print("\nPositive = more likely to open credit card")
print("Negative = less likely to open credit card\n")
print(feature_importance.head(10).to_string(index=False))

## Step 5: Save Model to Snowflake Model Registry

In [ ]:
from sklearn.pipeline import Pipeline
from snowflake.ml.registry import Registry
from snowflake.ml.model import type_hints

pipeline = Pipeline([
    ('scaler', scaler),
    ('classifier', model)
])

reg = Registry(session=session, database_name="NEXT_BEST_PRODUCT_DEMO", schema_name="FEATURES")

In [ ]:
sample_input = X_test.head(10)

mv = reg.log_model(
    model=pipeline,
    model_name="CREDIT_CARD_PROPENSITY_MODEL",
    version_name="V1",
    comment="V1 Logistic Regression model for predicting credit card adoption. Simple and interpretable for CDO/Data Engineer review.",
    metrics={
        "accuracy": float(accuracy),
        "precision": float(precision),
        "recall": float(recall),
        "auc_roc": float(auc_roc)
    },
    sample_input_data=sample_input,
    conda_dependencies=["scikit-learn"]
)

print(f"Model saved to registry!")
print(f"Model: NEXT_BEST_PRODUCT_DEMO.FEATURES.CREDIT_CARD_PROPENSITY_MODEL")
print(f"Version: V1")

In [ ]:
print("\nTesting model from registry...")
test_predictions = mv.run(X_test.head(5), function_name="predict_proba")
print("\nSample predictions (probability of opening credit card):")
test_predictions

## Step 6: Generate Propensity Scores for All Members

In [ ]:
all_eligible = df[df['OPENED_CREDIT_CARD_12M'] == 0][feature_columns]
all_proba = pipeline.predict_proba(all_eligible)[:, 1]

propensity_df = df[df['OPENED_CREDIT_CARD_12M'] == 0][['MEMBER_ID']].copy()
propensity_df['CREDIT_CARD_PROPENSITY'] = all_proba
propensity_df = propensity_df.sort_values('CREDIT_CARD_PROPENSITY', ascending=False)

print("TOP 20 MEMBERS MOST LIKELY TO OPEN A CREDIT CARD:")
print("="*50)
propensity_df.head(20)

In [ ]:
session.write_pandas(
    propensity_df,
    table_name="CREDIT_CARD_PROPENSITY_SCORES",
    database="NEXT_BEST_PRODUCT_DEMO",
    schema="FEATURES",
    auto_create_table=True,
    overwrite=True
)
print("Propensity scores saved to NEXT_BEST_PRODUCT_DEMO.FEATURES.CREDIT_CARD_PROPENSITY_SCORES")

## Summary

**What we built:**
1. Realistic dummy data for 5,000 credit union members
2. Feature engineering: demographics, account holdings, transaction behavior, existing products
3. Two models compared:
   - **Logistic Regression**: Simple, interpretable - great for explaining to stakeholders
   - **XGBoost**: Often more accurate - good when prediction performance is priority
4. Model saved to Snowflake Model Registry

**Key Insights:**
- Both models identify similar important features
- Logistic Regression coefficients show direction (positive/negative influence)
- XGBoost feature importance shows relative predictive power

**Next Steps:**
- Marketing can target the top N members with highest propensity scores
- Repeat this approach for other products (auto loans, personal loans, etc.)
- Combine multiple models to create a "Next Best Product" recommendation per member